In [1]:
'''
Comp551: Assignment 1: Question 3

@author: Anand Kamat
@id: 260773313
'''
#Importing Packages
import numpy as np
import csv
from numpy.linalg._umath_linalg import inv
import matplotlib.pyplot as plt
from numba.tests.npyufunc.test_ufunc import dtype
import math


In [2]:
################ Question 3: Real Life Dataset ####################################################

#Datasets
corpus = []

####
#    Loading Datasets
####

#Loading communities dataset
with open('./Datasets/communities.data', 'rb') as data_file:
    for data in data_file:
        corpus.append(data)
    del(data)    

#Defining the 2D array    
for i in range(len(corpus)): 
    corpus[i] = corpus[i].split(",")
    
    
#Deleting the community names as they are stated as not important
corpus = np.delete(corpus, 3, 1)


In [3]:
#Filling missing data with sample mean
sample_mean = np.zeros(len(corpus[0]))
sample_mean_count = np.zeros(len(corpus[0]))
for i in corpus:
    for j in range(len(i)):
        if(i[j] != '?'):
            sample_mean[j] += float(i[j])
            sample_mean_count[j] += 1

for i in range(len(sample_mean)): 
    sample_mean[i] = sample_mean[i]/sample_mean_count[i]

for i in corpus:
    for j in range(len(i)):
        if(i[j] == '?'):
            i[j] = sample_mean[j]

#Adding bias term to the input data
corpus = np.hstack(((np.ones(len(corpus), dtype=float)).reshape(-1,1), corpus))


In [4]:
#Lists for regularizing term            
lam_list = [] 
training_error, testing_error = [],[]
avg_regul_train, avg_regul_test = [],[]

In [19]:
for lam in range(0,30):
    print(lam)
    lamb = float(lam+1)/10
    lam_list.append(lamb)    
    regul_training_error, regul_testing_error = [],[]
    for i in range(5):        
        #Creating training set
        set1_X_train = np.array(np.vstack((corpus[0:i*(len(corpus)/5)], corpus[(i+1)*(len(corpus)/5): len(corpus)])), dtype = float)
        set1_Y_train = set1_X_train[:,[len(set1_X_train[0]) -1]]
        set1_X_train = np.delete(set1_X_train, (len(set1_X_train[0]) -1), 1)
        
        #Creating Test set
        set1_X_test = np.array(corpus[i*(len(corpus)/5): (i+1)*(len(corpus)/5)], dtype = float)
        set1_Y_test = set1_X_test[:,[len(set1_X_test[0]) -1]]
        set1_X_test = np.delete(set1_X_test, (len(set1_X_test[0]) -1), 1)
    
        #Creating an identity matrix to add the regularizing term
        identity_matrix = np.identity(len(set1_X_train[0]), dtype = float)
        
        if(lam == 0):
            #Learning parameters using linear Regression
            param = np.dot(np.dot(inv(np.dot(np.transpose(set1_X_train),  set1_X_train)), np.transpose(set1_X_train)) , (set1_Y_train))
            
            #Writing Params to csv_file
            file_name_str = "no_regul_params_set_" + str(i+1) +".csv"
            myFile = open(file_name_str, 'w')  
            with myFile:  
                writer = csv.writer(myFile)
                writer.writerows(param)
            
            #Calculating Training Mean Squared Error
            prediction_train = np.dot(set1_X_train, param)
            squared_train_sum = 0
             
            for i in range(0, len(prediction_train)):
                squared_train_sum += ((prediction_train[i] - set1_Y_train[i])**2)
                 
            set1_training_mse = (squared_train_sum)/(len(prediction_train))    
            
            
            #Calculating Testing Mean Squared Error
            prediction_valid = np.dot(set1_X_test, param)
            squared_valid_sum = 0
             
            for i in range(0, len(prediction_valid)):
                squared_valid_sum += ((prediction_valid[i] - set1_Y_test[i])**2)
                 
            set1_testing_mse = (squared_valid_sum)/(len(prediction_valid))
        
        #Learning parameters using Ridge Regression
        param_regul = np.dot(np.dot(inv(np.dot(np.transpose(set1_X_train),  set1_X_train) + lamb*identity_matrix), np.transpose(set1_X_train)) , (set1_Y_train))
        
        #Writing Params to csv_file
        regul_file_name_str = "regul_" + str(lamb) + "_params_set_" + str(i+1) +".csv"
        myFile = open(regul_file_name_str, 'w')  
        with myFile:  
            writer = csv.writer(myFile)
            writer.writerows(param_regul)
        
        #Calculating Training Mean Squared Error Using Ridge Regression
        prediction_regul_train = np.dot(set1_X_train, param_regul)
        squared_regul_train_sum = 0
         
        for i in range(0, len(prediction_regul_train)):
            squared_regul_train_sum += ((prediction_regul_train[i] - set1_Y_train[i])**2)
             
        set1_regul_training_mse = (squared_regul_train_sum)/(len(prediction_regul_train))    
        
        #Calculating Testing Mean Squared Error Using Ridge Regression
        prediction_regul_valid = np.dot(set1_X_test, param_regul)
        squared_regul_valid_sum = 0
         
        for i in range(0, len(prediction_regul_valid)):
            squared_regul_valid_sum += ((prediction_regul_valid[i] - set1_Y_test[i])**2)
             
        set1_regul_testing_mse = (squared_regul_valid_sum)/(len(prediction_regul_valid))
        
        regul_training_error.append(set1_regul_training_mse[0])
        regul_testing_error.append(set1_regul_testing_mse[0])
             
        #Finding Training Error and Testing Error 
        if(len(training_error) < 5):
            training_error.append(set1_training_mse[0])
            testing_error.append(set1_testing_mse[0])
    
    avg_regul_train.append(np.mean(regul_training_error))
    avg_regul_test.append(np.mean(regul_testing_error))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [6]:
mean_MSE = np.mean(testing_error)
# #print(testing_error)
# #print(lam_list)
#print(avg_regul_test)

In [7]:
testing_error_str = "Average testing error (without regularization) is: " + str(mean_MSE)
print(testing_error_str)
#print(mean_MSE)
#print(np.mean(training_error))

Average testing error (without regularization) is: 0.223086110832


In [30]:
MSE_string = []
for i in range(len(lam_list)-1):
    regul_str = "MSE (test data) when lambda = " + str(lam_list[i]) + " is: " + str(avg_regul_test[i])
    MSE_string.append("Lambda: " + str(lam_list[i]) + " MSE: " + str(avg_regul_test[i]))
    print(regul_str)

regul_file_name_str = "MSE with regularization.csv"
myFile = open(regul_file_name_str, 'w')  
with myFile:  
    writer = csv.writer(myFile)
    writer.writerow(MSE_string)

MSE (test data) when lambda = 0.1 is: 0.0187862773198
MSE (test data) when lambda = 0.2 is: 0.0186692281549
MSE (test data) when lambda = 0.3 is: 0.0186060680898
MSE (test data) when lambda = 0.4 is: 0.0185653399606
MSE (test data) when lambda = 0.5 is: 0.0185365970605
MSE (test data) when lambda = 0.6 is: 0.0185152488173
MSE (test data) when lambda = 0.7 is: 0.0184988990627
MSE (test data) when lambda = 0.8 is: 0.0184861416054
MSE (test data) when lambda = 0.9 is: 0.0184760793191
MSE (test data) when lambda = 1.0 is: 0.018468103582
MSE (test data) when lambda = 1.1 is: 0.0184617819032
MSE (test data) when lambda = 1.2 is: 0.0184567957487
MSE (test data) when lambda = 1.3 is: 0.0184529037923
MSE (test data) when lambda = 1.4 is: 0.0184499189952
MSE (test data) when lambda = 1.5 is: 0.0184476936667
MSE (test data) when lambda = 1.6 is: 0.0184461093611
MSE (test data) when lambda = 1.7 is: 0.0184450698317
MSE (test data) when lambda = 1.8 is: 0.0184444959877
MSE (test data) when lambda =

In [9]:
min_lambda_value = lam_list[avg_regul_test.index(min(avg_regul_test))]
lambda_str = "Value of lambda giving the smallest test MSE: " + str(min_lambda_value) + " and Average Testing MSE is: " + str(min(avg_regul_test))
print(lambda_str)

Value of lambda giving the smallest test MSE: 1.9 and Average Testing MSE is: 0.0184443222005
